# AUTOMATIC1111 Stable Diffusion WebUI 1.5 + ChilloutMix Checkpoint

More info: https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb

> Created by [@wibus-wee](https://github.com/wibus-wee)
>
> Reference: [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab)

In [ ]:
#@title 选择模型 { display-mode: "form" }
endpoint = 'https://civitai.com/api/v1/models'
#@markdown 选择模型
checkpointName = 'chilloutmix' #@param ["chilloutmix", "sunshinemix", "grapefruitHentaiModel"]
#@markdown 其他模型 （填入模型ID, 填入后会覆盖上面的选择，如 6424）
checkpointID = '' #@param {type:"string"}

defaultCheckpoint = {
    'chilloutmix': '6424',
    'sunshinemix': '9291',
    'grapefruitHentaiModel': '2583'
}

downloadId = checkpointID if checkpointID else defaultCheckpoint[checkpointName]

import requests
res = requests.get(endpoint + '/' + downloadId).json()
downloadLink = res['modelVersions'][0]['files'][0]['downloadUrl']
checkpoint = res['modelVersions'][0]['files'][0]['name']
print("已选择模型: " + checkpoint + " <===> " + downloadLink)

%store checkpoint
%store downloadLink

In [ ]:
#@title 2. 检查 GPU & 检查环境

#@markdown 此步骤将检查你的 GPU 是否支持 xformers，同时会检查是否为 Paperspace 平台（仅针对 M4000 GPU 做了判断，因此付费 GPU 可能在此处将会出现逻辑判断错误的情况，你可能需要自行勾选为 Paperspace 平台）。

import os, subprocess
paperspace_m4000 = False
#@markdown 是否为 Paperspace 平台
isPaperspace = False #@param {type:"boolean"}
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    if 'M4000' in subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8'):
        print("WARNING: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        paperspace_m4000 = True
        isPaperspace = True
    else:
        print("你正在使用的是合适的 GPU - " + subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8') + "。")
        print("平台: Paperspace" if isPaperspace else "使用平台: Colab")
except:
    print("似乎没有 GPU 可用。请检查你的运行时类型。")
    exit()

rootDir = isPaperspace and '/tmp' or '/content'
stableDiffusionWebUIInstalled = os.path.exists(rootDir + '/stable-diffusion-webui')
%store rootDir
%store paperspace_m4000 
%store isPaperspace
%store stableDiffusionWebUIInstalled

In [ ]:
#@title 3. 第一次使用 - 安装依赖并启动

#@markdown ## **Extensions**
#@markdown > 你可以在安装时选择安装扩展，扩展将会在启动时自动安装。如果你想要在启动后再安装扩展，你可能需要重启运行时。

#@markdown 安装 xformer 包 (仅在 Paperspace M4000 GPU 平台上运行时需要选择，若选择则会构建 xformers 包).
xformersInstall = True #@param {type:"boolean"}
#@markdown 安装 ControlNet 扩展
controlNetExtension = True #@param {type:"boolean"}
#@markdown 安装 OpenPose Editor 扩展
openPoseExtension = True #@param {type:"boolean"}
#@markdown 安装 Civitai Browser 扩展
civitaiBrowserExtension = True #@param {type:"boolean"}
#@markdown 安装 HuggingFace 扩展
huggingFaceExtension = True #@param {type:"boolean"}
#@markdown 安装 Images Browser 扩展
imagesBrowserExtension = True #@param {type:"boolean"}
#@markdown 安装 Additional Networks 扩展
additionalNetworksExtension = True #@param {type:"boolean"}
#@markdown 安装 Deforum 扩展
deforumExtension = False #@param {type:"boolean"}
#@markdown 安装 Kohya sd-scripts 扩展 ( 启动后再安装需要重启 )
kohyaExtension = False #@param {type:"boolean"}
#@markdown 安装 DreamBooth 扩展
dreamBoothExtension = False #@param {type:"boolean"}

#@markdown ---

#@markdown ## **LoRAs**
#@markdown > 你可以在安装时选择安装模型，模型将会在启动前自动安装。

#@markdown Korean Doll Likeness `v1.0`
koreanDollLikenessv10 = True #@param {type:"boolean"}
#@markdown Korean Doll Likeness `v1.5`
koreanDollLikenessv15 = True #@param {type:"boolean"}
#@markdown St. Louis (Luxurious Wheels) (Azur Lane)
stLouis = True #@param {type:"boolean"}
#@markdown Girls' Frontline-OTs-14"lightning"
girlsFrontlineOTs14Lightning = True #@param {type:"boolean"}
#@markdown Yae Miko | Realistic Genshin (Mixed)
yaeMikoRealisticGenshinMixed = True #@param {type:"boolean"}
#@markdown ChilloutMixss
chilloutMixss = True #@param {type:"boolean"}
#@markdown Fashion Girl
fashionGirl = True #@param {type:"boolean"}
#@markdown Cute Girl mix4
cuteGirlMix4 = True #@param {type:"boolean"}

loraDownloadIDs = {
  'koreanDollLikenessv10': 'https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb/releases/download/koreanDollLikeness/koreanDollLikeness_v10.safetensors',
  'koreanDollLikenessv15': 'https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb/releases/download/koreanDollLikeness/koreanDollLikeness_v15.safetensors',
  'stLouis': '6669',
  'girlsFrontlineOTs14Lightning': '6525',
  'yaeMikoRealisticGenshinMixed': '8484',
  'chilloutMixss': '10850',
  'fashionGirl': {
    "id": 8217,
    "version": "v4.5"
  },
  'cuteGirlMix4': '14171',
}

def getLatestLoraDownloadURL(id):
  if id.isdigit():
    import requests
    res = requests.get(endpoint + '/' + id).json()
    if res["baseModel"] != "SD 1.5":
      print("Lora model " + id + " is not SD 1.5, may not work. Skip.")
      return None
    downloadLink = res['modelVersions'][0]['files'][0]['downloadUrl']
    name = res['modelVersions'][0]['files'][0]['name']
    return {
      'url': downloadLink,
      'name': name
    }
  else:
    return {
      'url': id,
      'name': id.split('/')[-1]
    }

def getSpecificLoraDownloadURL(id, version):
  if id.isdigit():
    import requests
    res = requests.get(endpoint + '/' + id).json()
    if res["baseModel"] != "SD 1.5":
      print("Lora model " + id + " is not SD 1.5, may not work. Skip.")
      return None
    for modelVersion in res['modelVersions']:
      if modelVersion['name'] == version:
        downloadLink = modelVersion['files'][0]['downloadUrl']
        name = modelVersion['files'][0]['name']
        return {
          'url': downloadLink,
          'name': name
        }
  else:
    return {
      'url': id,
      'name': id.split('/')[-1]
    }

def getLoraDownloadURLs():
  downloadURLs = []
  for key in loraDownloadIDs:
    if not eval(key): # skip if not selected
      continue
    if type(loraDownloadIDs[key]) is int:
      downloadURLs.append(getLatestLoraDownloadURL(loraDownloadIDs[key]))
    elif type(loraDownloadIDs[key]) is dict: # {id: 123, version: 'v1.0'}
      downloadURLs.append(getLatestLoraDownloadURL(loraDownloadIDs[key]['id']))
    elif type(loraDownloadIDs[key]) is str: # url
      downloadURLs.append(loraDownloadIDs[key])
  downloadURLs = [x for x in downloadURLs if x is not None]
  downloadURLs = list(dict.fromkeys(downloadURLs))
  return downloadURLs

loraDownloadURLs = getLoraDownloadURLs()

%store -r paperspace_m4000 
%store -r isPaperspace
%store -r rootDir 
%store -r checkpoint 
%store -r downloadLink
%store -r stableDiffusionWebUIInstalled

import os, subprocess
from glob import glob
from pathlib import Path
import sys

!apt-get -y install -qq aria2
ariaInstalled = False

try:
    subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
    ariaInstalled = True
except:
    pass

if paperspace_m4000:
  if xformersInstall:
    !pip install ninja
    !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers
    !pip install -q --pre triton
else:
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  !pip install -q --pre triton
  

!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui {rootDir}/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {rootDir}/stable-diffusion-webui/scripts/run_n_times.py
if deforumExtension:
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {rootDir}/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
if imagesBrowserExtension:
  !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
if huggingFaceExtension:
  !git clone https://github.com/camenduru/stable-diffusion-webui-huggingface {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
if civitaiBrowserExtension:
  !git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser {rootDir}/stable-diffusion-webui/extensions/sd-civitai-browser
if openPoseExtension:
  !git clone https://github.com/camenduru/openpose-editor {rootDir}/stable-diffusion-webui/extensions/openpose-editor
if controlNetExtension:
  !git clone https://github.com/Mikubill/sd-webui-controlnet {rootDir}/stable-diffusion-webui/extensions/sd-webui-controlnet
if additionalNetworksExtension:
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks
if kohyaExtension:
  !git clone https://github.com/ddpn08/kohya-sd-scripts-webui.git {rootDir}/stable-diffusion-webui/extensions/kohya-sd-scripts-webui
if dreamBoothExtension:
  !git clone https://github.com/d8ahazard/sd_dreambooth_extension {rootDir}/stable-diffusion-webui/extensions/sd_dreambooth_extension

if isPaperspace:
  %cd /stable-diffusion-webui
else:
  %cd {rootDir}/stable-diffusion-webui


webuiControlNetModels = [
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors",
]
annotatorLink = [
  "https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth",
]

def ariaDownload(downloadLink, checkpoint, path):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {downloadLink} -d {path} -o {checkpoint}
def wgetDownload(downloadLink, checkpoint, path):
  !wget -c {downloadLink} -P {path} -O {checkpoint}  

if ariaInstalled:
    ariaDownload(downloadLink, checkpoint, rootDir + "/stable-diffusion-webui/models/Stable-diffusion")
    if controlNetExtension:
      for model in webuiControlNetModels:
        ariaDownload(model, {model.split('/')[-1]}, rootDir + "/stable-diffusion-webui/extensions/sd-webui-controlnet/models")
      for model in annotatorLink:
        ariaDownload(model, {model.split('/')[-1]}, rootDir + "/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator")
    for model in loraDownloadURLs:
      ariaDownload(model.url, model.name, rootDir + "/stable-diffusion-webui/models/Lora")
else:
    wgetDownload(downloadLink, checkpoint, rootDir + "/stable-diffusion-webui/models/Stable-diffusion")
    if controlNetExtension:
      for model in webuiControlNetModels:
        wgetDownload(model, {model.split('/')[-1]}, rootDir + "/stable-diffusion-webui/extensions/sd-webui-controlnet/models")
      for model in annotatorLink:
        wgetDownload(model, {model.split('/')[-1]}, rootDir + "/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator")
    for model in loraDownloadURLs:
      wgetDownload(model.url, model.name, rootDir + "/stable-diffusion-webui/models/Lora")

if additionalNetworksExtension:
  !ln -s {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora {rootDir}/stable-diffusion-webui/models/Lora


stableDiffusionWebUIInstalled = True
%store stableDiffusionWebUIInstalled

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
if dreamBoothExtension:
  !export REQS_FILE="./extensions/sd_dreambooth_extension/requirements.txt"
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}

In [ ]:

#@title 4. 重启 - 重新启动 Stable Diffusion WebUI

%store -r paperspace_m4000 
%store -r rootDir 
%store -r stableDiffusionWebUIInstalled

if not stableDiffusionWebUIInstalled:
    print("请先运行安装 Stable Diffusion WebUI 的单元格。")
    exit()

if additionalNetworksExtension:
  !ln -s {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora {rootDir}/stable-diffusion-webui/models/Lora

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}

In [ ]:
#@title 5. 导出 - 导出生成的图片
%store -r rootDir 

from pathlib import Path
import os, subprocess

export_storage_dir = Path(rootDir, 'export')
export_storage_dir.mkdir(exist_ok=True)

!if [ $(dpkg-query -W -f='${Status}' p7zip-full 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y p7zip-full; fi # install 7z if it isn't already installed
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H-%M-%S')
%cd "{export_storage_dir}"
!mkdir -p "{datetime_str}/log"
!cd "{rootDir}/stable-diffusion-webui/log" && mv * "{export_storage_dir / datetime_str / 'log'}"
!cd "{rootDir}/stable-diffusion-webui/outputs" && mv * "{export_storage_dir / datetime_str}"
s = subprocess.run(f'find "{Path(export_storage_dir, datetime_str)}" -type d -name .ipynb_checkpoints -exec rm -rv {{}} +', shell=True)
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{datetime_str}.7z" "{export_storage_dir / datetime_str}"